# Reccomendation systems and matrix factorization: simple problem

In [5]:
import numpy as np
from numba import jit, cuda

Goal: predict rating matrix R

In [6]:
@jit(target_backend='cuda')
def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    '''
    R: rating matrix
    P: |U| * K (User features matrix)
    Q: |D| * K (Item features matrix)
    K: latent features
    steps: iterations
    alpha: learning rate
    beta: regularization parameter'''
    Q = Q.T

    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    # calculate error
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])

                    for k in range(K):
                        # calculate gradient with a and beta parameter
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])

        eR = np.dot(P,Q)

        e = 0

        for i in range(len(R)):

            for j in range(len(R[i])):

                if R[i][j] > 0:

                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)

                    for k in range(K):

                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        # 0.001: local minimum
        if e < 0.001:

            break

    return P, Q.T

In [7]:
R = [

     [5,3,0,1],

     [4,0,0,1],

     [1,1,0,5],

     [1,0,0,4],

     [0,1,5,4],
    
     [2,1,3,0],

    ]

R = np.array(R)
# N: num of User
N = len(R)
# M: num of Movie
M = len(R[0])
# Num of Features
K = 4

print(N, ' users, ', M, ' movies, ', K ,' latent features. \n')

 
P = np.random.rand(N,K)
Q = np.random.rand(M,K)

 

nP, nQ = matrix_factorization(R, P, Q, K)

print(nP,'\n',nQ)

nR = np.dot(nP, nQ.T)

6  users,  4  movies,  4  latent features. 

[[ 2.21372298  0.77463189  0.49124813  0.3633514 ]
 [ 1.56210693  0.26626723  0.82584016  0.45431807]
 [-0.17240097  1.70757604  0.71960777  1.07517046]
 [-0.11541815  0.92258779  0.50144412  1.35555145]
 [ 0.42723794  1.10282556  0.666133    1.29368385]
 [ 0.75908983  0.34217763  0.12135414  0.75895846]] 
 [[ 1.93393304  0.16811448  0.83969293  0.45959907]
 [ 1.08752885  0.52033128  0.27531648 -0.03380285]
 [ 1.59791576  1.50249961  0.89434836  1.53923227]
 [-0.52591091  1.5231627   0.87067812  1.53141119]]


In [8]:
print(R)
print(nR)

[[5 3 0 1]
 [4 0 0 1]
 [1 1 0 5]
 [1 0 0 4]
 [0 1 5 4]
 [2 1 3 0]]
[[4.99091238 2.9335192  5.69985611 0.99982872]
 [3.96802988 2.04939369 4.33407152 0.99882796]
 [1.05205322 0.86279024 4.58866721 4.9646585 ]
 [0.97595865 0.44676504 3.73673358 3.97845406]
 [2.16557356 1.17813541 4.92672009 4.01624301]
 [1.97627073 1.01133366 3.00382959 1.38991645]]
